In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd


In [ ]:
#proposed workflow: have all individual variables have the MSTN varaible to match, merge each of them to a parent dataframe, starting with passengermiles
#get log of all the variables
#

# AM Dataframe

In [ ]:
# read in the required dataframes for the model: passenger miles traveled, peak fare per track mile, auto travel time in min/mile
# bus travel time in min/mile, households in .5 miles, parking capacity, bus line count, jobs in .5 miles, trains per hour in the peak, terminal station dummy

am_passengers = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\ridership_AM.xlsx")
fare_track_miles = pd.read_excel() #need to run
auto_tt_minpmile = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\Auto _ bus travel time+M25 station\25_35_drive_times_distances_updated.xlsx")
bus_tt_minpmile = pd.read_csv() #need to run
households_half_mile = pd.read_excel() 
AM_parking capacity = pd.read_excel() #need to run
num_bus_lines = pd.read_csv(r"C:\Users\janna\Box\WMATA_DATA\zheng_results\bus_line_stop.csv")
jobs_half_mile = pd.read_excel() #need to run
trains_per_hour = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\Ridership data\avgTrainThroughput_byPeriod_PeakOnly.xlsx") 
terminal_station = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\zheng_results\metro_ternimal_dummy.csv")
track_miles = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\Ridership data\trackmile_parkingusers_\railOD_trackMiles_spring2022.xlsx", header=1, index_col=0)

#mstns to merge easily
mstns = pd.read_excel(r"C:\Users\janna\Box\WMATA_DATA\Ridership data\mstn_id_to_stn_name.xlsx")

## AM Passenger Miles

In [14]:
#make track miles into OD pairs
track_miles = track_miles.stack().rename_axis(('O',"D")).reset_index(name='track_miles')

                 O                   D  value
0     Addison Road        Addison Road   0.10
1     Addison Road           Anacostia  10.35
2     Addison Road            Archives   8.31
3     Addison Road  Arlington Cemetery  11.39
4     Addison Road         Ballston-MU  14.27
...            ...                 ...    ...
8276  Woodley Park    West Hyattsville   9.74
8277  Woodley Park             Wheaton  14.38
8278  Woodley Park         White Flint   9.62
8279  Woodley Park              Wiehle  22.92
8280  Woodley Park        Woodley Park   0.10

[8281 rows x 3 columns]


In [17]:
#give station names proper MSTN IDs
track_miles2 = track_miles.merge(mstns, left_on = 'O', right_on= 'PRIMARY_NAME_FY23')
track_miles2 = track_miles.merge(mstns, left_on = 'D', right_on= 'PRIMARY_NAME_FY23')

In [ ]:
# create an od_pair key column
am_passengers['pairs'] = am_passengers.apply(lambda x: str(x['ID'])+str(0)+str(x['ID_1']), axis=1)
track_miles2['pairs'] = track_miles2.apply(lambda x: str(x['ID_x'])+str(0)+str(x['ID_y']), axis=1)


In [20]:
#merge track miles to passengers in the am, multiply the track miles by passengers, consolidate the columns, rename to passenger column
milesmerge = track_miles2.merge(am_passengers, left_on='pairs', right_on='pairs')
milesmerge['riders_miles'] = milesmerge['track_miles'] * milesmerge['COUNT']
am_dataframe_new = milesmerge[['O', 'D', 'track_miles', 'ID_x', 'ID_y','pairs', 'COUNT', 'ridersmiles', ]]
am_dataframe_new.rename(columns={'COUNT':'passengers'})

,O,D,value,ID_x,PRIMARY_NAME_FY23_x,ID_y,PRIMARY_NAME_FY23_y,pairs,Unnamed: 0,ID,ID_1,START_PLACE_NAME,END_PLACE_NAME,COUNT,riders_miles
0,Anacostia,Addison Road,10.35,MSTN_001,Anacostia,MSTN_062,Addison Road,MSTN_0010MSTN_062,59,MSTN_001,MSTN_062,Anacostia,Addison Road,9,93.15
1,Ballston-MU,Addison Road,14.27,MSTN_068,Ballston-MU,MSTN_062,Addison Road,MSTN_0680MSTN_062,5526,MSTN_068,MSTN_062,Ballston,Addison Road,8,114.16
2,Benning Road,Addison Road,2.46,MSTN_003,Benning Road,MSTN_062,Addison Road,MSTN_0030MSTN_062,211,MSTN_003,MSTN_062,Benning Road,Addison Road,56,137.76
3,Bethesda,Addison Road,16.36,MSTN_052,Bethesda,MSTN_062,Addison Road,MSTN_0520MSTN_062,4286,MSTN_052,MSTN_062,Bethesda,Addison Road,7,114.52
4,Braddock Road,Addison Road,15.00,MSTN_078,Braddock Road,MSTN_062,Addison Road,MSTN_0780MSTN_062,6406,MSTN_078,MSTN_062,Braddock Road,Addison Road,4,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6171,Vienna,Woodley Park,17.27,MSTN_069,Vienna,MSTN_040,Woodley Park,MSTN_0690MSTN_040,5593,MSTN_069,MSTN_040,Vienna,Woodley Park-Zoo,105,1813.35
6172,Waterfront,Woodley Park,4.46,MSTN_039,Waterfront,MSTN_040,Woodley Park,MSTN_0390MSTN_040,3223,MSTN_039,MSTN_040,Waterfront,Woodley Park-Zoo,10,44.60
6173,West Falls Church,Woodley Park,12.38,MSTN_059,West Falls Church,MSTN_040,Woodley Park,MSTN_0590MSTN_040,4839,MSTN_059,MSTN_040,West Falls Church,Woodley Park-Zoo,9,111.42
6174,West Hyattsville,Woodley Park,9.74,MSTN_055,West Hyattsville,MSTN_040,Woodley Park,MSTN_0550MSTN_040,4519,MSTN_055,MSTN_040,West Hyattsville,Woodley Park-Zoo,17,165.58


## AM Trains per Hour

In [47]:
#zhila calculated it without separating the origins and the destinations
trains_per_hour = trains_per_hour[ (trains_per_hour['ARRIVAL_PERIOD']=='AM Peak') & (trains_per_hour['SERVICETYPE']=='Weekday')]
trains_per_hour

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,ARRIVAL_PERIOD,AVG_TRAINS
8,MSTN_001,F06,1,Weekday,AM Peak,17
10,MSTN_001,F06,2,Weekday,AM Peak,16
20,MSTN_002,F02,1,Weekday,AM Peak,31
22,MSTN_002,F02,2,Weekday,AM Peak,31
32,MSTN_003,G01,1,Weekday,AM Peak,24
...,...,...,...,...,...,...
1126,MSTN_090,N04,2,Weekday,AM Peak,14
1136,MSTN_091,N06,1,Weekday,AM Peak,14
1138,MSTN_091,N06,2,Weekday,AM Peak,13
1148,MSTN_098,C11,1,Weekday,AM Peak,30


In [48]:
trains_per_hour = trains_per_hour.groupby(['STATION_ID']).sum().reset_index()
trains_per_hour

C:\Users\janna\AppData\Local\Temp\ipykernel_6280\187997880.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  trainsam_o = trainsam_o.groupby(['STATION_ID']).sum().reset_index()


,STATION_ID,DIR_ID,AVG_TRAINS
0,MSTN_001,3,33
1,MSTN_002,3,62
2,MSTN_003,3,51
3,MSTN_004,3,56
4,MSTN_005,3,78
...,...,...,...
87,MSTN_088,3,27
88,MSTN_089,3,27
89,MSTN_090,3,28
90,MSTN_091,3,27


In [ ]:
am_dataframe = am_dataframe.merge(trains_per_hour, left_on='MSTN_ID_O', right_on='STATION_ID')
am_dataframe = am_dataframe.merge(trains_per_hour, left_on='MSTN_ID_D', right_on='STATION_ID')
am_dataframe.head()

## Auto Travel Time in Minutes/Mile

In [ ]:
auto_tt_minpmile